In [1]:
# A couple requried imports
from glob import glob 
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
from tifffile import imread, TiffFile
from skimage import io


# Importing the deep learning framework
from csbdeep.models import CARE, Config

from csbdeep.utils import plot_some, plot_history, Path

Using TensorFlow backend.


In [2]:
base_dir = r'X:\GautamDey\2020\Fast Nup imaging\separate for denoising'
base_path = Path(base_dir)
C1_path = Path(base_path/r'C1'/r'raw data')
C2_path = Path(base_path/r'C2'/r'raw data')

C1_files = sorted(C1_path.rglob('*.tif'))
C2_files = sorted(C2_path.rglob('*.tif'))

save_dir_C1 = C1_path/f'results'
save_dir_C1.mkdir(exist_ok=True)
save_dir_C2 = C2_path/f'results'
save_dir_C2.mkdir(exist_ok=True)

print(f'There are {len(C1_files)} channel 1 files and {len(C2_files)} channel 2 files')

There are 11 channel 1 files and 11 channel 2 files


In [3]:
C1_ims = []
C1_names = []
C2_ims = []
C2_names = []
for file in tqdm(C1_files):
    C1_names.append(file.stem)
    T1 = imread(str(file))
    C1_ims.append(T1)
for file in tqdm(C2_files):
    C2_names.append(file.stem)
    T2 = imread(str(file))
    C2_ims.append(T2)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:24<00:00,  2.21s/it]


In [4]:
model1 = CARE(config=None, name='noise2noise_C1_norm', basedir='models')
model2 = CARE(config=None, name='noise2noise_cut11', basedir='models')

W0317 07:28:07.381232 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0317 07:28:07.384311 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0317 07:28:07.407856 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0317 07:28:07.440747 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0317 07:28:07.445755 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\t

Loading network weights from 'weights_best.h5'.


W0317 07:28:07.804584 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0317 07:28:07.804584 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

W0317 07:28:07.820207 13388 module_wrapper.py:139] From C:\Users\SC\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



Loading network weights from 'weights_best.h5'.


In [5]:
def normalize(img, mean, std):
    return (img - mean)/std

def denormalize(img, mean, std):
    return (img * std) + mean

In [6]:
counter = 0
for T in tqdm(C1_ims):
    denoised_1 = []
    for frame in T:
        mean = np.mean(frame)
        std = np.std(frame)
        pred = denormalize(model1.predict(normalize(frame, mean, std), axes='YX', normalizer=None), mean, std)
        denoised_1.append(pred)
    io.imsave(f'{save_dir_C1}/{C1_names[counter]}.tif', np.asarray(denoised_1))
    counter+=1

counter = 0
for T in tqdm(C2_ims):
    denoised_2 = []
    for frame in T:
        mean = np.mean(frame)
        std = np.std(frame)
        pred = denormalize(model2.predict(normalize(frame, mean, std), axes='YX', normalizer=None), mean, std)
        denoised_2.append(pred)
    io.imsave(f'{save_dir_C2}/{C2_names[counter]}.tif', np.asarray(denoised_2))
    counter+=1

100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [22:43<00:00, 123.93s/it]


In [ ]:
C2_names = []
#for file in C1_files:
#    C1_names.append(file.stem)
#    T1 = imread(str(file))
#    C1_ims.append(T1)
for file in tqdm(C2_files):
    C2_names.append(file.stem)